In [258]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import pandas as pd
from typing import Counter

In [259]:
def load_and_prepare_data(seed=42):
    dataset = load_dataset("dair-ai/emotion")
    train_validation = dataset["train"].train_test_split(test_size=0.25, seed=seed)
    train_data = train_validation["train"]
    validation_data = train_validation["test"]
    test_data = dataset["test"]
    return train_data, validation_data, test_data

train_data, validation_data, test_data = load_and_prepare_data()


In [260]:
def extract_features_labels(data):
    texts = data["text"]
    labels = data["label"]
    return texts, pd.Series(labels)

X_train, y_train = extract_features_labels(train_data)
X_val, y_val = extract_features_labels(validation_data)
X_test, y_test = extract_features_labels(test_data)

In [261]:
def compute_class_weights(y_train):
    label_counts = Counter(y_train)
    num_classes = len(label_counts)
    total_samples = sum(label_counts.values())
    class_weights = [
        total_samples / (num_classes * label_counts[i])
        for i in range(num_classes)
    ]
    return class_weights

class_weights = compute_class_weights(y_train)
class_weights

[0.5704506560182544,
 0.4973887092762994,
 2.0181634712411705,
 1.2338062924120914,
 1.3956734124214933,
 4.672897196261682]

In [262]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    min_df=.001,
)

tfidf = vectorizer.fit_transform(X_train)
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df

,ability,able,absolutely,abused,accept,acceptable,accepted,ache,aching,act,...,wrong,wronged,wrote,www,year,years,yes,yesterday,young,youre
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
11996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
11997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
11998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [263]:
X_train_v = vectorizer.fit_transform(X_train)
X_val_v = vectorizer.transform(X_val)
X_test_v = vectorizer.transform(X_test)

In [264]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print(class_weight_dict)

sample_weights = [class_weight_dict[label] for label in y_train]


{np.int64(0): np.float64(0.5704506560182544), np.int64(1): np.float64(0.4973887092762994), np.int64(2): np.float64(2.0181634712411705), np.int64(3): np.float64(1.2338062924120914), np.int64(4): np.float64(1.3956734124214933), np.int64(5): np.float64(4.672897196261682)}


In [271]:
xgb_model = XGBClassifier(
    objective='multi:softprob',
    num_class=6,
    eval_metric='mlogloss',
    n_estimators=20000,
    learning_rate=0.001,
    random_state=42,
    #use gpu cuda
    # tree_method='gpu_hist', 
    device='cuda'
 )

xgb_model.fit(X_train_v, y_train, eval_set=[(X_val_v, y_val)], verbose=500,sample_weight=sample_weights)

[0]	validation_0-mlogloss:1.79146
[500]	validation_0-mlogloss:1.68166
[1000]	validation_0-mlogloss:1.60539
[1500]	validation_0-mlogloss:1.54758
[2000]	validation_0-mlogloss:1.50514
[2500]	validation_0-mlogloss:1.47204
[3000]	validation_0-mlogloss:1.44477
[3500]	validation_0-mlogloss:1.42344
[4000]	validation_0-mlogloss:1.40689
[4500]	validation_0-mlogloss:1.39341
[5000]	validation_0-mlogloss:1.38145
[5500]	validation_0-mlogloss:1.36974
[6000]	validation_0-mlogloss:1.35989
[6500]	validation_0-mlogloss:1.35373
[7000]	validation_0-mlogloss:1.34976
[7500]	validation_0-mlogloss:1.34649
[8000]	validation_0-mlogloss:1.34354
[8500]	validation_0-mlogloss:1.34169
[9000]	validation_0-mlogloss:1.34085
[9500]	validation_0-mlogloss:1.34058
[10000]	validation_0-mlogloss:1.34068
[10500]	validation_0-mlogloss:1.34125
[11000]	validation_0-mlogloss:1.34240
[11500]	validation_0-mlogloss:1.34370
[12000]	validation_0-mlogloss:1.34507
[12500]	validation_0-mlogloss:1.34646
[13000]	validation_0-mlogloss:1.3478

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,'cuda'
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [277]:
def calc_accuracy(y_true, y_pred):
    correct = sum(y_true == y_pred)
    total = len(y_true)
    return correct / total

In [279]:
from sklearn.metrics import classification_report
classes = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
target_names = [classes[i] for i in range(len(classes))]

y_pred = xgb_model.predict(X_test_v)
accuracy = calc_accuracy(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))


Test Accuracy: 0.7595
Classification Report:
              precision    recall  f1-score   support

     sadness       0.93      0.72      0.81       581
         joy       0.64      0.94      0.76       695
        love       0.72      0.69      0.71       159
       anger       0.97      0.63      0.76       275
        fear       0.91      0.57      0.70       224
    surprise       0.69      0.61      0.65        66

    accuracy                           0.76      2000
   macro avg       0.81      0.69      0.73      2000
weighted avg       0.81      0.76      0.76      2000



In [283]:
label_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
def predict_sentiment_examples(sentences, xgb_model: XGBClassifier, vectorizer: TfidfVectorizer):
    # print("sentence:",sentence)
    X_input = vectorizer.transform(sentences)  # shape: (1, n_features)
    # print("X_input:", X_input.shape)
    pred = xgb_model.predict(X_input)        # extract scalar
    # print("preds:", pred)
    probs = xgb_model.predict_proba(X_input) # shape: (num_classes,)
    # print("probs:", probs)
    return pred, np.max(probs, axis=1)


In [ ]:
def predict_sentiment_one_example(sentence, xgb_model: XGBClassifier, vectorizer: TfidfVectorizer):
    X_input = vectorizer.transform([sentence])  
    pred = xgb_model.predict(X_input)      
    probs = xgb_model.predict_proba(X_input) 
    return label_map[pred[0]], np.max(probs, axis=1)[0]


In [286]:
example_texts = [
    "iam very sad i lost my job and i dont know what to do i have no money and i have no friends",
    "i love you",
    "iam very upset iam sick",
    "i cant stand delaying my PhD defense any more",
    "i can punch them in the face right now",
    "the hardest part about growing up is saying goodby to childhood dreams",
    "how did you do that to me?",
    "how dare you stand where he stod?"
]
for text in example_texts:
    sentiment, prob = predict_sentiment_one_example(text, xgb_model, vectorizer)
    print(f"Text: {text}\nPredicted: {sentiment} ({prob:.3f})\n")

Text: iam very sad i lost my job and i dont know what to do i have no money and i have no friends
Predicted: sadness (0.218)

Text: i love you
Predicted: joy (0.186)

Text: iam very upset iam sick
Predicted: joy (0.186)

Text: i cant stand delaying my PhD defense any more
Predicted: joy (0.186)

Text: i can punch them in the face right now
Predicted: joy (0.186)

Text: the hardest part about growing up is saying goodby to childhood dreams
Predicted: joy (0.186)

Text: how did you do that to me?
Predicted: joy (0.186)

Text: how dare you stand where he stod?
Predicted: joy (0.186)



In [ ]:

expanded_texts = [
    "iam very sad i lost my job and i dont know what to do i have no money and i have no friends",
    "It genuinely brightens my day to finally meet you — I’ve been looking forward to this moment for so long.",
    "I love you more deeply than words can express — your presence brings peace to my chaos and warmth to my coldest days.",
    "I feel completely drained and overwhelmed — being sick like this makes everything feel heavier, and I just want to curl up and disappear for a while.",
    "I’m beyond frustrated with how long this delay has dragged on — every day feels like a slap in the face to the effort I’ve poured into this PhD.",
    "I’m so furious I could scream — the way they treated me was completely disrespectful, and I honestly feel like punching them in the face.",
    "Growing up feels like slowly letting go of the dreams that once defined me — saying goodbye to those innocent hopes is the hardest part of all.",
    "After everything we’ve been through, how could you betray me like that? I trusted you, and now I feel completely shattered.",
    "How dare you stand where he stood, pretending like you belong — you have no right to be here after everything you’ve done."
]


preds, probs = predict_sentiment_examples(expanded_texts, xgb_model, vectorizer)
for text, pred, prob in zip(expanded_texts, preds, probs):
    sentiment = label_map[pred]
    print(f"Text: {text}\nPredicted Sentiment: {sentiment} (Confidence: {prob:.3f})\n")


Text: iam very sad i lost my job and i dont know what to do i have no money and i have no friends
Predicted Sentiment: sadness (Confidence: 0.218)

Text: It genuinely brightens my day to finally meet you — I’ve been looking forward to this moment for so long.
Predicted Sentiment: joy (Confidence: 0.186)

Text: I love you more deeply than words can express — your presence brings peace to my chaos and warmth to my coldest days.
Predicted Sentiment: joy (Confidence: 0.186)

Text: I feel completely drained and overwhelmed — being sick like this makes everything feel heavier, and I just want to curl up and disappear for a while.
Predicted Sentiment: sadness (Confidence: 0.242)

Text: I’m beyond frustrated with how long this delay has dragged on — every day feels like a slap in the face to the effort I’ve poured into this PhD.
Predicted Sentiment: joy (Confidence: 0.186)

Text: I’m so furious I could scream — the way they treated me was completely disrespectful, and I honestly feel like punc

In [288]:
import gradio as gr

# Gradio wrapper
def gradio_predict(text):
    sentiment, prob = predict_sentiment_one_example(text, xgb_model, vectorizer)
    return f"Predicted Emotion: {sentiment} ({prob:.3f} confidence)"

# Launch Gradio interface
gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(lines=4, placeholder="Enter a sentence to analyze..."),
    outputs="text",
    title="Emotion Classifier",
    description="Enter a sentence and get its predicted emotion using XGBoost and TF-IDF/BERT features.",
    examples=[[text] for text in example_texts]

).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [276]:
import winsound
winsound.Beep(1000, 500)  # Beep sound to indicate completion